In [1]:
import numpy as np
import torch
from torch.autograd import Variable
from torch import nn
import loader
from threes import *
from qLearningNet import *
import time

MOVES = [0, 1, 2, 3]
CUDA = False

FILENAME = "saved_parameters"
INPUT_SIZE = 19
HIDDEN_SIZE = 256
GAMMA = 0.8


# parametry sieci np. learning rate
# lepszy zbior uczacy - jak w DeepMindzie
# nagrody - musza byc wieksze - inaczej sami sobie redukujemy learning rate
# (gestsze nagrody - np. logarytm z rewardow)
# uczyc jednak dobrymi przebiegami
# http://pytorch.org/docs/master/nn.html#embedding 
# napisac do Adriana po GCP

In [5]:
def train(model, data_loaders, optimizer, num_epochs=500, log_every=100, verbose=True):
    if CUDA:
        model.network.cuda()

    data = np.zeros((1000,40))
    data_ptr = 0
    
    iter_ = 0
    epoch = 0
    if verbose:
        print u'Training the model!'
        print u'Interrupt at any time to get current model'
    try:
        while epoch < num_epochs:
            epoch += 1
            x = data_loaders.get(model, 30)
            #data[data_ptr:data_ptr + 30] = x
            #data_ptr += 30
            #data_ptr %= 1000 - 30
            
            #random_data_ptr = np.random.randint(0,data_ptr)
            #x = data[random_data_ptr : random_data_ptr + 30]
            
            future = x[:, 21:]
            future_scores = model.Q(future)
            for i, row in enumerate(future):
                #print(row)
                game = Threes(save_game=False, data=row.tolist())
                for j, move in enumerate(MoveEnum):
                    if not game.canMove(move):
                        future_scores[i, j] = float('-inf')
                if not game.getPossibleMoves():
                    future_scores[i:,:] = np.full((1,4), x[i, 20])
                
            y = x[:, 20] + GAMMA * np.max(future_scores, axis=1)
            xx = x[:, :19]
            iter_ += 1

            optimizer.zero_grad()
            i = np.asarray(np.vstack((np.arange(0,30),x[:,19])),int)
            
            i = torch.LongTensor(i)
            out = model.Q(xx, as_variable=True)[i[0],i[1]]
            loss = model.loss(out, y)
            loss.backward()
            optimizer.step()

            if iter_ % log_every == 0 and verbose:
                print u"Minibatch {0: >6}  | loss {1: >15.12f} ".format(iter_, loss.data[0])

    except KeyboardInterrupt:
        pass
    model.save_parameters(FILENAME)


q_learning_net = QLearningNet()
for p in q_learning_net.network.parameters():
    p.requires_grad = True
optimizer = torch.optim.Adam(q_learning_net.network.parameters(), lr=0.001)
data_loader = loader.Loader()
train(q_learning_net, data_loader, optimizer, num_epochs=10000)
print("Learning done")

Training the model!
Interrupt at any time to get current model
Minibatch    100  | loss  0.000000132858 
Minibatch    200  | loss  0.000000080590 
Minibatch    300  | loss  0.000000140362 
Minibatch    400  | loss  0.000003429838 
Minibatch    500  | loss  0.000006068040 
Minibatch    600  | loss  0.000000958801 
Minibatch    700  | loss  0.000000036021 
Minibatch    800  | loss  0.000253256207 
Minibatch    900  | loss  0.000000229461 
Minibatch   1000  | loss  0.000001666498 
Minibatch   1100  | loss  0.000000706156 
Minibatch   1200  | loss  0.000000108188 
Minibatch   1300  | loss  0.000002521189 
Minibatch   1400  | loss  0.000000749856 
Minibatch   1500  | loss  0.000000084256 
Minibatch   1600  | loss  0.000000263931 
Minibatch   1700  | loss  0.000018225184 
Minibatch   1800  | loss  0.000000357961 
Minibatch   1900  | loss  0.000002630846 
Minibatch   2000  | loss  0.000001175052 
Minibatch   2100  | loss  0.000023824326 
Minibatch   2200  | loss  0.000000170629 
Minibatch   2

In [3]:
from AIModels import *
game = Threes(save_game=False)
ai = QLearningNetAI(game, FILENAME)
scores, move_count, highs = AIModel.test_ai(ai, 100, verbose=False)
print move_count
print np.mean(scores), np.mean(highs)

Loading net parameters
{'Down': 1051, 'Right': 750, 'Up': 824, 'Left': 796}
192.15 25.38


In [10]:
game = Threes(save_game=False)
ai = RandomAI(game)
scores, move_count, highs = AIModel.test_ai(ai, 100, verbose=False)
print move_count
print np.mean(scores), np.mean(highs)

{'Down': 1107, 'Right': 1090, 'Up': 1034, 'Left': 1119}
298.26 35.22


In [11]:
game = Threes(save_game=False)
ai = MiniMaxAI(game)
scores, move_count, highs = AIModel.test_ai(ai, 10, verbose=False)
print move_count
print np.mean(scores), np.mean(highs)

{'Down': 775, 'Right': 741, 'Up': 761, 'Left': 779}
12808.5 412.8
